# AutoGluon: Porto Seguro (Classification) # 4

Based on: https://www.linkedin.com/learning/automl-build-production-ready-models-quickly/

Changed metric from "roc_auc" to AUCPR (like H20)? "average_precision"?

Use AUCRP as stopping metric? "average_precision"?

In [1]:
time_limit = 60*60  # 1 hour max (confirm with timer)
metric = "average_precision"  # specify your evaluation metric
stopping = "average_precision"  # specify your stopping metric
label = "target" # name of target or label variable (just happens to be called "target" for this dataset)

## Import

In [2]:
# Import packages

from autogluon.tabular import TabularDataset
from autogluon.tabular import TabularPredictor
# from sklearn.metrics import average_precision_score  # same as AUCPR (?) - implemented as "average_precision"

import numpy as np
import pandas as pd

In [3]:
# Import data

train_data = TabularDataset("porto_train.csv")
test_data = TabularDataset("porto_test.csv")

In [4]:
train_data.shape

(476170, 60)

In [5]:
test_data.shape

(119042, 59)

## Tidy

In [6]:
# Replace "-1" with missing value
train_data = train_data.replace(-1, np.nan)
test_data = test_data.replace(-1, np.nan)

## Transform

In [7]:
# Drop variable not used as predictors
train_data = train_data.drop(["id", "fold"], axis=1)
test_data = test_data.drop(["id"], axis=1)

In [8]:
# Convert categorical variables to "category" data type
cat_vars = [col for col in train_data.columns if 'cat' in col]
cat_vars

['ps_ind_02_cat',
 'ps_ind_04_cat',
 'ps_ind_05_cat',
 'ps_car_01_cat',
 'ps_car_02_cat',
 'ps_car_03_cat',
 'ps_car_04_cat',
 'ps_car_05_cat',
 'ps_car_06_cat',
 'ps_car_07_cat',
 'ps_car_08_cat',
 'ps_car_09_cat',
 'ps_car_10_cat',
 'ps_car_11_cat']

In [9]:
for col in cat_vars:
    test_data[col] = test_data[col].astype('category')
    
cat_vars = cat_vars + ["target"]

for col in cat_vars:
    train_data[col] = train_data[col].astype('category')

In [10]:
# Create separate test data to demonstrate how to make predictions on new examples at inference time:
y_test = test_data[label]  # values to predict

## Visualize

(skipped)

## Model

In [11]:
%%time

# gets stuck at
# Fitting model: CatBoost_BAG_L1 ... Training model for up to 3304.05s of the 3304.04s of remaining time.
#     Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy
# so removed from fit
#     refit_full='best'
# crashed so killed all "jobs" and retried

predictor = TabularPredictor(
    label=label, 
    eval_metric=metric
).fit(
    train_data, 
    presets=['best_quality'], 
    time_limit=time_limit,
    refit_full='best',
    ag_args_fit={"stopping_metric": stopping}
    #stopping_metric=stopping
)

No path specified. Models will be saved in: "AutogluonModels/ag-20230804_135009/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20230804_135009/"
AutoGluon Version:  0.7.0
Python Version:     3.9.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Jun 20 11:48:01 UTC 2023
Train Data Rows:    476170
Train Data Columns: 57
Label Column: target
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Gene

	Stopping at the best epoch learned earlier - 6.
	93.98s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: XGBoost_BAG_L1_FULL ...
	7.96s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: NeuralNetTorch_BAG_L1_FULL ...
	50.91s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: LightGBMLarge_BAG_L1_FULL ...
	2.82s	 = Training   runtime
Fitting model: WeightedEnsemble_L2_FULL | Skipping fit via cloning parent ...
	153.86s	 = Training   runtime
Refit complete, total runtime = 272.83s
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20230804_135009/")


CPU times: user 3h 49min 59s, sys: 3min 27s, total: 3h 53min 27s
Wall time: 40min 10s


In [12]:
# Use trained models to make predictions on the new data and then evaluate performance:
y_pred = predictor.predict_proba(test_data)

In [13]:
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

/home/stever7/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Evaluation: average_precision on test data: 0.06705443214518564
Evaluations on test data:
{
    "average_precision": 0.06705443214518564,
    "accuracy": 0.9635842811780716,
    "balanced_accuracy": 0.5,
    "mcc": 0.0,
    "roc_auc": 0.6366034321897859,
    "f1": 0.0,
    "precision": 0.0,
    "recall": 0.0
}


In [14]:
%%time

# Evaluate the performance of each individual trained model on test data:
predictor.leaderboard(test_data, silent=True)

CPU times: user 25min 51s, sys: 17.4 s, total: 26min 8s
Wall time: 1min 7s


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.067054,0.067665,35.695313,135.178497,1785.138277,0.016658,0.135370,153.858140,2,True,14
1,CatBoost_BAG_L1,0.067038,0.066590,1.358117,1.427846,597.727660,1.358117,1.427846,597.727660,1,True,7
2,WeightedEnsemble_L2_FULL,0.066846,NaN,11.391772,NaN,448.313114,0.015701,NaN,153.858140,2,True,24
3,XGBoost_BAG_L1,0.066475,0.065053,7.115191,4.751624,17.407671,7.115191,4.751624,17.407671,1,True,11
4,NeuralNetFastAI_BAG_L1,0.066248,0.064030,10.532800,5.725078,447.282962,10.532800,5.725078,447.282962,1,True,10
5,CatBoost_BAG_L1_FULL,0.066232,NaN,0.197803,NaN,74.018290,0.197803,NaN,74.018290,1,True,18
6,XGBoost_BAG_L1_FULL,0.064659,NaN,0.689396,NaN,7.962955,0.689396,NaN,7.962955,1,True,21
7,NeuralNetFastAI_BAG_L1_FULL,0.064538,NaN,1.378824,NaN,93.977010,1.378824,NaN,93.977010,1,True,20
8,LightGBMXT_BAG_L1_FULL,0.064045,NaN,0.137888,NaN,3.113669,0.137888,NaN,3.113669,1,True,15
9,LightGBM_BAG_L1,0.063956,0.063094,0.989887,1.508925,16.656536,0.989887,1.508925,16.656536,1,True,4


In [15]:
predictor.get_model_best()

'WeightedEnsemble_L2'

In [16]:
%%time

test_leaderboard = predictor.leaderboard(extra_info=True, silent=True)

CPU times: user 50.6 s, sys: 19 s, total: 1min 9s
Wall time: 49.3 s


In [17]:
test_leaderboard.T

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
model,WeightedEnsemble_L2,CatBoost_BAG_L1,XGBoost_BAG_L1,NeuralNetFastAI_BAG_L1,LightGBMXT_BAG_L1,LightGBM_BAG_L1,NeuralNetTorch_BAG_L1,LightGBMLarge_BAG_L1,RandomForestEntr_BAG_L1,ExtraTreesGini_BAG_L1,...,RandomForestGini_BAG_L1_FULL,ExtraTreesEntr_BAG_L1_FULL,RandomForestEntr_BAG_L1_FULL,XGBoost_BAG_L1_FULL,WeightedEnsemble_L2_FULL,NeuralNetTorch_BAG_L1_FULL,NeuralNetFastAI_BAG_L1_FULL,LightGBMXT_BAG_L1_FULL,LightGBMLarge_BAG_L1_FULL,CatBoost_BAG_L1_FULL
score_val,0.067665,0.06659,0.065053,0.06403,0.063636,0.063094,0.062679,0.061499,0.057205,0.056639,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pred_time_val,135.178497,1.427846,4.751624,5.725078,1.50886,1.508925,5.112818,1.476186,38.868979,39.469021,...,37.454506,38.71723,38.868979,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fit_time,1785.138277,597.72766,17.407671,447.282962,123.95472,16.656536,367.454471,15.802125,23.091878,14.31349,...,23.936477,14.622172,23.091878,7.962955,448.313114,50.914193,93.97701,3.113669,2.818332,74.01829
pred_time_val_marginal,0.13537,1.427846,4.751624,5.725078,1.50886,1.508925,5.112818,1.476186,38.868979,39.469021,...,37.454506,38.71723,38.868979,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fit_time_marginal,153.85814,597.72766,17.407671,447.282962,123.95472,16.656536,367.454471,15.802125,23.091878,14.31349,...,23.936477,14.622172,23.091878,7.962955,153.85814,50.914193,93.97701,3.113669,2.818332,74.01829
stack_level,2,1,1,1,1,1,1,1,1,1,...,1,1,1,1,2,1,1,1,1,1
can_infer,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
fit_order,14,7,11,10,3,4,12,13,6,8,...,16,19,17,21,24,22,20,15,23,18
num_features,9,57,57,57,57,57,57,57,57,57,...,57,57,57,57,9,57,57,57,57,57
